In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import pandas as pd
from dev_interaction_util import *
from DevCvAnalysis import DevCvAnalysis

In [25]:
config = load_config("config.yml") 

dropbox_data_dir = config['dropbox_data_dir']


The basic structure is the following:

1. Run the following cross-validated analyses:
   1. Predicting change by condition only
   2. Predicting change by condition and neural and behavioral measures
   3. Predicting change by condition, neural and behavioral measures, and their interactions
2. Measure the predictivity of the three models above using anova
3. Repeat the steps above separately for three outcome variables, change in: FFQ, ASA-24, and BFP
4. 

Tasks to do to get this job done (not in order):

1. Write the analysis pipeline above
2. Get the neural data
3. Get the behavioral data


We have the behavioral data. Do we have the neural data already?

What could we delegate here? Behavioral data we already have. We have mostly writen the analysis pipeline. The neural data could be passed on.

# Get the data

## Set up data


In [26]:
%load_ext autoreload
%autoreload 2

from InterventionCVManager import *

dropbox_data_dir = config['dropbox_data_dir']

icvm = InterventionCVManager(dropbox_data_dir)
#icvm.mode = 'full_pipeline_test'
#icvm.mode = 'fast_pipeline_test'
icvm.mode = 'full_analysis'

#dev_cv_analysis = icvm.get_prepopulated_dev_cv_analysis(set_as_random=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
%load_ext autoreload
%autoreload 2
dev_cv_analysis = icvm.get_prepopulated_dev_cv_analysis(set_as_random=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV032', 'DEV007', 'DEV002', 'DEV280', 'DEV022'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(243, 33)
(243, 33)


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


# Predict change

### Set up sets of variables to run

First, we set up a function that loops runs the scoring loop above (which does one cross-validation analysis), and the nadditionally:
- selects the best model based on the overall results
- Runs a final fit
- presents model results

Now, we include functions that compare models with and without individual differences and interactions.

Should manually verify that in the following list, the intervention_group allocations are randomized (if we're running a test run) or that they are accurate (if it's not a test run).

In [28]:
pd.concat([
    dev_cv_analysis.outcome_measures['SID'],
      dev_cv_analysis.group_assignments
],axis=1)

,SID,intervention_group
0,DEV004,umpqua
1,DEV005,umpqua
2,DEV008,umpqua
3,DEV009,umpqua
4,DEV010,mckenzie
...,...,...
238,DEV308,willamette
239,DEV309,umpqua
240,DEV310,mckenzie
241,DEV311,willamette


In [29]:
dev_cv_analysis.outcome_measures.columns

Index(['SID', 'bf', 'NUTRIENT_DENSITY_2wkAverage',
       'ANTINUTRIENT_DENSITY_2wkAverage', 'total_calorie'],
      dtype='object')

In [30]:
condition_cols = dev_cv_analysis.group_assignment_onehots.columns.tolist()
inddiff_cols = dev_cv_analysis.get_predictors_main_names()

interaction_cols = [id + "*" + cond for id in inddiff_cols for cond in condition_cols]

predictor_sets = {
    'condition_only': condition_cols,
    'condition_inddiff': condition_cols + inddiff_cols,
    'condition_inddiff_interactions': condition_cols + inddiff_cols + interaction_cols
}


outcome_vars_to_try = [ 'bf','cancer_promoting_FFQ',
       'NUTRIENT_DENSITY_2wkAverage']

In [31]:
print(", ".join(dev_cv_analysis.get_predictor_data().columns))

EDM, BIS_11, PCS, ACES_sum, BFI_agreeableness, BFI_conscientiousness, BFI_extraversion, BFI_neuroticism, BFI_openness, NCS_total, TESQ_E_sum, SRHI_healthy_minus_unhealthy, RTFS_f1_minus_f2, cancer_promoting_minus_preventing_FCI, age365, education_own, household_income_per_person, SST_PostErrorSlowW1_mean, SST_mean_ssrt_0, ROC_Crave_Regulate_Minus_Look, ROC_Crave_Minus_Neutral, WTP_unhealthy_minus_healthy, wtp_liked_value_association-test_z_FDR_0.01, roc_reappraiseCrave_reappraisal_association-test_z_FDR_0.01, roc_reappraiseCrave_multivariate_regulation, sst_CorrectGo_striatum_joint_mask, sst_FailedStop_motor_control_striatum_joint_mask, sst_CorrectGoFollowingFailedStop_striatum_joint_mask, Planning_aggregate, Restraint_aggregate, IMI_effort_importance_aggregate, wtp_roc_koban_kober_craving_combined, birthsex_factor_Male, umpqua, mckenzie, EDM*umpqua, BIS_11*umpqua, PCS*umpqua, ACES_sum*umpqua, BFI_agreeableness*umpqua, BFI_conscientiousness*umpqua, BFI_extraversion*umpqua, BFI_neurotic

In [32]:
print(", ".join(dev_cv_analysis.outcome_measures.columns))

SID, bf, NUTRIENT_DENSITY_2wkAverage, ANTINUTRIENT_DENSITY_2wkAverage, total_calorie


## Total calories

In [33]:
model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'total_calorie', dev_cv_analysis)

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV032', 'DEV007', 'DEV002', 'DEV280', 'DEV022'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(243, 33)
(243, 33)
 attempting to predict total_calorie with 2 predictors in the set condition_only
predictors in that set are umpqua mckenzie
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', StandardScaler(

doing permutation test on importance; this may take time.


...

Time elapsed for Lasso: 82.61 seconds
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', SVR())])
{'estimator__kernel': ['linear'], 'estimator__C': [0.01, 0.1, 1, 10, 100]}
Fitting 4 folds for each of 5 candidates, totalling 20 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', SVR())])
{'feature_selection__score_func': [<function f_regression at 0x181be9b20>], 'feature_selection__k': [5, 10, 15], 'estimator__kernel': ['linear'], 'estimator__C': [0.01, 0.1, 1, 10, 100]}
Fitting 4 folds for each of 15 candidates, totalling 60 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', RFE(estimator=LinearRegression())),
                ('estimator', SVR())])
{'feature_selection__n_features_to_select': [5, 10, 15], 'feature_selection__step': [5], 'estimator__kernel': ['linear'], 'estimator__C': [0.01, 0.1, 1, 10, 100]}
Fitting 4 folds for each of 15 candidates, totalling 60 fits
Time

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


doing permutation test on importance; this may take time.


,predictor,coef,feature_importance,fa_abs
22,ROC_Crave_Minus_Neutral,None,0.027028,0.027028
9,BFI_neuroticism,None,0.025836,0.025836
15,cancer_promoting_minus_preventing_FCI,None,0.025618,0.025618
23,WTP_unhealthy_minus_healthy,None,0.024670,0.024670
28,sst_FailedStop_motor_control_striatum_joint_mask,None,0.023311,0.023311
13,SRHI_healthy_minus_unhealthy,None,0.022967,0.022967
97,wtp_roc_koban_kober_craving_combined*umpqua,None,0.016475,0.016475
87,sst_FailedStop_motor_control_striatum_joint_mask*umpqua,None,0.015572,0.015572
81,roc_reappraiseCrave_reappraisal_association-test_z_FDR_0.01*umpqua,None,0.015261,0.015261
61,cancer_promoting_minus_preventing_FCI*umpqua,None,0.015073,0.015073


{'condition_only': -0.027994679083579756, 'condition_inddiff': -0.18965578086905355, 'condition_inddiff_interactions': -0.03040357767162778}


## ANTINUTRIENT_DENSITY_2wkAverage

In [34]:
model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'ANTINUTRIENT_DENSITY_2wkAverage', dev_cv_analysis)

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV032', 'DEV007', 'DEV002', 'DEV280', 'DEV022'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(243, 33)
(243, 33)
 attempting to predict ANTINUTRIENT_DENSITY_2wkAverage with 2 predictors in the set condition_only
predictors in that set are umpqua mckenzie
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler

doing permutation test on importance; this may take time.
Number of selected features: 0


...

Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Ridge())])
{'feature_selection__score_func': [<function f_regression at 0x181be9b20>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', RFE(estimator=LinearRegression())),
                ('estimator', Ridge())])
{'feature_selection__n_features_to_select': [5, 10, 15], 'feature_selection__step': [5], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Time elapsed for Ridge: 51.18 seconds
Pipeline(steps=[('scaler', S

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


doing permutation test on importance; this may take time.
Number of selected features: 1


,predictor,coef,feature_importance,fa_abs
89,sst_CorrectGoFollowingFailedStop_striatum_joint_mask*umpqua,0.280675,0.005904,0.005904
0,umpqua,-0.000000,0.000000,0.000000
35,EDM*umpqua,-0.000000,0.000000,0.000000
38,BIS_11*mckenzie,0.000000,0.000000,0.000000
39,PCS*umpqua,-0.000000,0.000000,0.000000
43,BFI_agreeableness*umpqua,-0.000000,0.000000,0.000000
51,BFI_openness*umpqua,-0.000000,0.000000,0.000000
52,BFI_openness*mckenzie,0.000000,0.000000,0.000000
53,NCS_total*umpqua,-0.000000,0.000000,0.000000
55,TESQ_E_sum*umpqua,-0.000000,0.000000,0.000000


{'condition_only': -0.026705836214731705, 'condition_inddiff': -0.108865217812564, 'condition_inddiff_interactions': -0.03794594106196829}


## NUTRIENT_DENSITY_2wkAverage

In [35]:
model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'NUTRIENT_RICH_FOODS_INDEX_2wkAverage', dev_cv_analysis)

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV032', 'DEV007', 'DEV002', 'DEV280', 'DEV022'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(243, 33)
(243, 33)
 attempting to predict NUTRIENT_DENSITY_2wkAverage with 2 predictors in the set condition_only
predictors in that set are umpqua mckenzie
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', S

doing permutation test on importance; this may take time.
Number of selected features: 2


...

Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', RFE(estimator=LinearRegression())),
                ('estimator', Lasso())])
{'feature_selection__n_features_to_select': [5, 10, 15], 'feature_selection__step': [5], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Time elapsed for Lasso: 61.74 seconds
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', SVR())])
{'estimator__kernel': ['linear'], 'estimator__C': [0.01, 0.1, 1, 10, 100]}
Fitting 4 folds for each of 5 candidates, totalling 20 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', SVR())])
{'feature_selection__score_func': [<function f_regression at 0x181be9b20>], 'feature_selection__k': [5, 10, 15], 'estimator__kernel': ['linear'], 'estimator__C': [0.01, 0.1, 1, 10, 100]}
Fitting 4 folds for each of 15 candidates, totalling 60 fits
Pipeline(ste

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


doing permutation test on importance; this may take time.
Number of selected features: 10


,predictor,coef,feature_importance,fa_abs
42,ACES_sum*mckenzie,1.449145,0.025430,0.025430
91,Planning_aggregate*umpqua,0.739333,0.007863,0.007863
94,Restraint_aggregate*mckenzie,-0.835481,0.007591,0.007591
35,EDM*umpqua,0.552099,0.004342,0.004342
73,ROC_Crave_Regulate_Minus_Look*umpqua,-0.378407,0.003079,0.003079
95,IMI_effort_importance_aggregate*umpqua,0.360889,0.002348,0.002348
37,BIS_11*umpqua,0.381820,0.002310,0.002310
43,BFI_agreeableness*umpqua,-0.078163,-0.000214,0.000214
93,Restraint_aggregate*umpqua,0.019615,0.000042,0.000042
47,BFI_extraversion*umpqua,-0.023823,-0.000036,0.000036


{'condition_only': -0.004378070583131932, 'condition_inddiff': -0.026744715870021717, 'condition_inddiff_interactions': -0.029128539120856334}


## Body Fat Percentage

In [36]:



model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'bf', dev_cv_analysis)

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV032', 'DEV007', 'DEV002', 'DEV280', 'DEV022'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(243, 33)
(243, 33)
 attempting to predict bf with 2 predictors in the set condition_only
predictors in that set are umpqua mckenzie
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', StandardScaler()),
       

doing permutation test on importance; this may take time.


...

Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Ridge())])
{'feature_selection__score_func': [<function f_regression at 0x181be9b20>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', RFE(estimator=LinearRegression())),
                ('estimator', Ridge())])
{'feature_selection__n_features_to_select': [5, 10, 15], 'feature_selection__step': [5], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Time elapsed for Ridge: 36.74 seconds
Pipeline(steps=[('scaler', S

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


doing permutation test on importance; this may take time.


,predictor,coef,feature_importance,fa_abs
67,household_income_per_person*umpqua,None,0.027706,0.027706
79,wtp_liked_value_association-test_z_FDR_0.01*umpqua,None,0.025488,0.025488
97,wtp_roc_koban_kober_craving_combined*umpqua,None,0.022318,0.022318
81,roc_reappraiseCrave_reappraisal_association-test_z_FDR_0.01*umpqua,None,0.021318,0.021318
3,BIS_11,None,0.019453,0.019453
68,household_income_per_person*mckenzie,None,0.019002,0.019002
84,roc_reappraiseCrave_multivariate_regulation*mckenzie,None,0.018331,0.018331
100,birthsex_factor_Male*mckenzie,None,0.018212,0.018212
66,education_own*mckenzie,None,0.018066,0.018066
62,cancer_promoting_minus_preventing_FCI*mckenzie,None,0.017030,0.017030


{'condition_only': -0.03203212616417565, 'condition_inddiff': -0.08707355783435684, 'condition_inddiff_interactions': -0.1463168816171625}


## Nutrient density

In [37]:


model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'NUTRIENT_DENSITY_2wkAverage')

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV032', 'DEV007', 'DEV002', 'DEV280', 'DEV022'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(243, 33)
(243, 33)
 attempting to predict NUTRIENT_DENSITY_2wkAverage with 2 predictors in the set condition_only
predictors in that set are umpqua mckenzie
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', S

doing permutation test on importance; this may take time.
Number of selected features: 2


...

Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', RFE(estimator=LinearRegression())),
                ('estimator', Lasso())])
{'feature_selection__n_features_to_select': [5, 10, 15], 'feature_selection__step': [5], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 21 candidates, totalling 84 fits
Time elapsed for Lasso: 54.76 seconds
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', SVR())])
{'estimator__kernel': ['linear'], 'estimator__C': [0.01, 0.1, 1, 10, 100]}
Fitting 4 folds for each of 5 candidates, totalling 20 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', SVR())])
{'feature_selection__score_func': [<function f_regression at 0x181be9b20>], 'feature_selection__k': [5, 10, 15], 'estimator__kernel': ['linear'], 'estimator__C': [0.01, 0.1, 1, 10, 100]}
Fitting 4 folds for each of 15 candidates, totalling 60 fits
Pipeline(ste

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


doing permutation test on importance; this may take time.
Number of selected features: 10


,predictor,coef,feature_importance,fa_abs
42,ACES_sum*mckenzie,1.449145,0.018393,0.018393
94,Restraint_aggregate*mckenzie,-0.835481,0.012204,0.012204
91,Planning_aggregate*umpqua,0.739333,0.007302,0.007302
35,EDM*umpqua,0.552099,0.003970,0.003970
73,ROC_Crave_Regulate_Minus_Look*umpqua,-0.378407,0.003056,0.003056
37,BIS_11*umpqua,0.381820,0.002845,0.002845
95,IMI_effort_importance_aggregate*umpqua,0.360889,0.002272,0.002272
43,BFI_agreeableness*umpqua,-0.078163,-0.000270,0.000270
47,BFI_extraversion*umpqua,-0.023823,-0.000067,0.000067
93,Restraint_aggregate*umpqua,0.019615,0.000059,0.000059


{'condition_only': -0.004378070583131932, 'condition_inddiff': -0.026744715870021717, 'condition_inddiff_interactions': -0.029128539120856334}
